# <center>Data mining for consumer knowledge<center>

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as func
from pyspark.sql.window import Window
from pyspark.sql.functions import split, col, expr, udf, concat_ws
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

In [2]:
spark = SparkSession.builder.appName("consumers_knowledge_discovery2").getOrCreate()

In [3]:
demographics = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___DEMOGRAPHICS\demographics.csv")
smartwatch = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___MEDICAL\smart_watch\smart_watch.csv")
heart_rate = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___MEDICAL\heart_rate\heart_rate.csv")
symptoms = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___MEDICAL\searched_symptoms\symptoms.csv")
mental_health = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___MEDICAL\mental_health_corpus\mental_health.csv")
dating_app = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___PSYCHOGRAPHICS\dating_app\dating_app.csv")
reviews = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___PSYCHOGRAPHICS\reviews\reviews.csv")
reviews = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\___PSYCHOGRAPHICS\stress_level___\stress_level.csv")
ads = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____BROWSER\click on add prediction\click_ads.csv")
cookies = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____BROWSER\cookies\cookies.csv")
logs = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____BROWSER\logs\weblog.csv")
web_history = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____BROWSER\web_history\p_history\web_history.csv")
bank_loan = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____FINANCIAL\bank_loan\bank_loan.csv")
credit_card = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____FINANCIAL\credit_card\credit_card.csv")
online_shoppings = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\Public\DW\____FINANCIAL\online_retail\online_retail.csv")

### Marital status, age distribution, education, occupation and income information

In [8]:
demographics.select("marital-status").groupBy("marital-status").count().orderBy(func.desc("count")).show(10)
demographics.select("age").groupBy("age").count().orderBy(func.desc("count")).show(10)
demographics.select("education").groupBy("education").count().orderBy(func.desc("count")).show(10)
demographics.select("occupation").groupBy("occupation").count().orderBy(func.desc("count")).show(10, truncate = False)
demographics.select("salary").groupBy("salary").count().orderBy(func.desc("count")).show(10)

+--------------+-----+
|marital-status|count|
+--------------+-----+
|       Married| 4691|
| Never-married| 3311|
|      Divorced| 1385|
|     Separated|  321|
|       Widowed|  292|
+--------------+-----+

+---+-----+
|age|count|
+---+-----+
| 31|  293|
| 33|  277|
| 35|  277|
| 34|  272|
| 25|  272|
| 23|  271|
| 37|  270|
| 30|  267|
| 32|  258|
| 20|  257|
+---+-----+
only showing top 10 rows

+------------+-----+
|   education|count|
+------------+-----+
|     HS-grad| 3232|
|Some-college| 2305|
|   Bachelors| 1630|
|     Masters|  531|
|   Assoc-voc|  416|
|        11th|  363|
|  Assoc-acdm|  318|
|        10th|  295|
|     7th-8th|  205|
| Prof-school|  174|
+------------+-----+
only showing top 10 rows

+-----------------+-----+
|occupation       |count|
+-----------------+-----+
|Prof-specialty   |1257 |
|Exec-managerial  |1211 |
|Craft-repair     |1207 |
|Adm-clerical     |1187 |
|Sales            |1179 |
|Other-service    |1028 |
|Machine-op-inspct|621  |
|?                

### Religion & sexual orientation

In [10]:
result2 = dating_app.select("religion").groupBy("religion").count().orderBy(func.desc("count"))
if result2 is not None:
    windowSpec = Window.orderBy(func.desc("count"))
    result_with_row_number = result2.withColumn("row_number", func.row_number().over(windowSpec))
    filtered_result = result_with_row_number.filter("row_number > 1").limit(10)
    filtered_result.select('religion', 'count').show(truncate = False)
else:
    print("No valid result obtained.")
    
dating_app.select("orientation").groupBy("orientation").count().orderBy(func.desc("count")).show(10)

+-----------------------------------------+-----+
|religion                                 |count|
+-----------------------------------------+-----+
|agnosticism                              |472  |
|other                                    |439  |
|agnosticism but not too serious about it |424  |
|agnosticism and laughing about it        |403  |
|atheism                                  |384  |
|catholicism but not too serious about it |382  |
|atheism and laughing about it            |343  |
|other and laughing about it              |342  |
|christianity but not too serious about it|332  |
|christianity                             |318  |
+-----------------------------------------+-----+

+-----------+-----+
|orientation|count|
+-----------+-----+
|   straight| 8605|
|        gay|  944|
|   bisexual|  451|
+-----------+-----+



### Smartphone provider, preferred phone models & favorite time range for browsing the internet (relying on cookies)

In [11]:
cookies.select("manufacturer_name").groupBy("manufacturer_name").count().orderBy(func.desc("count")).show(10, truncate = False)
cookies.select("model_name").groupBy("model_name").count().orderBy(func.desc("count")).show(10)
cookies.select("part_of_day").groupBy("part_of_day").count().orderBy(func.desc("count")).show(10)

+---------------------------------------------------+-----+
|manufacturer_name                                  |count|
+---------------------------------------------------+-----+
|Apple                                              |3830 |
|Samsung                                            |2531 |
|Huawei                                             |1952 |
|Xiaomi                                             |1435 |
|Oppo                                               |48   |
|Realme Chongqing Mobile Telecommunications Corp Ltd|46   |
|Vivo                                               |38   |
|Huawei Device Company Limited                      |37   |
|Sony                                               |13   |
|Asus                                               |13   |
+---------------------------------------------------+-----+
only showing top 10 rows

+----------------+-----+
|      model_name|count|
+----------------+-----+
|       iPhone 11|  916|
|       iPhone XR|  462|
|        

### Bank loan duration & amount; purpose of bank loan

In [12]:
bank_loan.select("duration", "credit amount").describe().show()
bank_loan.select("purpose").groupBy("purpose").count().orderBy(func.desc("count")).show(10)

+-------+------------------+-----------------+
|summary|          duration|    credit amount|
+-------+------------------+-----------------+
|  count|              1000|             1000|
|   mean|            20.903|         3271.258|
| stddev|12.058814452756371|2822.736875960441|
|    min|                 4|              250|
|    max|                72|            18424|
+-------+------------------+-----------------+

+-------------------+-----+
|            purpose|count|
+-------------------+-----+
|                car|  337|
|        electronics|  280|
|furniture/equipment|  181|
|           business|   97|
|          education|   59|
|            repairs|   22|
|    vacation/others|   12|
|domestic appliances|   12|
+-------------------+-----+



### Alcohol consumption & diet habits

In [14]:
dating_app.select("drinks").groupBy("drinks").count().orderBy(func.desc("count")).show(10)

result = dating_app.select("diet").groupBy("diet").count().orderBy(func.desc("count"))
if result is not None:
    windowSpec = Window.orderBy(func.desc("count"))
    result_with_row_number = result.withColumn("row_number", func.row_number().over(windowSpec))
    filtered_result = result_with_row_number.filter("row_number > 1").limit(10)
    filtered_result.select('diet', 'count').show()
else:
    print("No valid result obtained.")

+-----------+-----+
|     drinks|count|
+-----------+-----+
|   socially| 7014|
|     rarely|  957|
|      often|  854|
|       null|  532|
| not at all|  527|
| very often|   62|
|desperately|   54|
+-----------+-----+

+-------------------+-----+
|               diet|count|
+-------------------+-----+
|    mostly anything| 2761|
|           anything| 1113|
|  strictly anything|  867|
|  mostly vegetarian|  602|
|       mostly other|  167|
|strictly vegetarian|  141|
|         vegetarian|  116|
|     strictly other|   65|
|              other|   57|
|       mostly vegan|   55|
+-------------------+-----+



### Individuals with excessive eating habits & individuals who eat too little

In [17]:
# Z-score
mean_val = smartwatch.selectExpr('avg(calories)').collect()[0][0]
stddev_val = smartwatch.selectExpr('stddev(calories)').collect()[0][0]

df_with_zscore = smartwatch.withColumn('zscore', (col('calories') - mean_val) / stddev_val)

# Filtering the outliers
upper_outliers = df_with_zscore.filter(col('zscore') > 3)
lower_outliers = df_with_zscore.filter(col('zscore') < -3)

upper_outliers.select('user_id', 'calories').show()
lower_outliers.select('user_id', 'calories').show()

+-------+--------+
|user_id|calories|
+-------+--------+
|     20|    4552|
|     20|    4546|
|     21|    4900|
|     32|    4547|
+-------+--------+

+-------+--------+
|user_id|calories|
+-------+--------+
|     25|     120|
|     28|       0|
|     30|       0|
+-------+--------+



### Finding neighbors (relying on zipcode) 

In [15]:
demographics.select("zipcode").groupBy("zipcode").count().orderBy(func.desc("count")).show(10)

+-------+-----+
|zipcode|count|
+-------+-----+
|  85323|    4|
|  57566|    4|
|  78226|    4|
|  45359|    4|
|  98148|    4|
|  68007|    4|
|   5089|    4|
|  38359|    3|
|  67556|    3|
|  74052|    3|
+-------+-----+
only showing top 10 rows



### Individuals who are most likely sick and unable to move, restricted to bed rest

In [18]:
smartwatch.createOrReplaceTempView("smartwatch")
results = smartwatch.select("user_id", "total_distance").where(func.col("total_distance") == 0)
results.show(10)

+-------+--------------+
|user_id|total_distance|
+-------+--------------+
|      4|           0.0|
|     12|           0.0|
|     12|           0.0|
|     13|           0.0|
|     18|           0.0|
|     20|           0.0|
|     21|           0.0|
|     21|           0.0|
|     21|           0.0|
|     21|           0.0|
+-------+--------------+
only showing top 10 rows



#### Individuals who are highly likely to be infected with COVID

In [19]:
symptoms.createOrReplaceTempView("symptoms")
query = symptoms.filter(col("symptoms").like("%tiredness%")) \
    .filter(col("symptoms").like("%high_fever%")) \
    .filter(col("symptoms").like("%cough%")) \
    .filter(col("symptoms").like("%loss_of_taste%"))
query.select('user_id', 'symptoms').show(truncate = False)

+-------+---------------------------------------------------------+
|user_id|symptoms                                                 |
+-------+---------------------------------------------------------+
|70     |['tiredness', ' high_fever', ' loss_of_taste', 'cough']  |
|71     |[' high_fever', ' loss_of_taste', ' cough', ' tiredness']|
|72     |['cough', ' high_fever', ' tiredness', ' loss_of_taste'] |
|73     |['loss_of_taste', ' tiredness', ' high_fever', ' cough'] |
|74     |[' high_fever', ' cough', ' tiredness', ' loss_of_taste']|
|77     |['loss_of_taste', ' cough', ' tiredness', 'high_fever']  |
|78     |[' high_fever', ' tiredness', ' cough', ' loss_of_taste']|
+-------+---------------------------------------------------------+



### Favourite faceboook & youtube pages of particular consumers

In [20]:
web_history.createOrReplaceTempView("web_history")
average_frequency = web_history.selectExpr("AVG(frequency)").collect()[0][0]
query = web_history.filter((col("user_id") == 1) &
                           (col("url").like("%facebook%")) &
                           (col("frequency") > average_frequency)) \
                   .orderBy(col("frequency").desc())
query.show()

+-------+--------------------+----------------+---------------+-----------+---------+-------+
|hist_id|                 url|     first_visit|     last_visit|click_count|frequency|user_id|
+-------+--------------------+----------------+---------------+-----------+---------+-------+
|      1|https://web.faceb...|  3/17/2023 1:12|3/29/2023 22:30|        177|    56640|      1|
|     18|https://web.faceb...| 3/13/2023 10:09| 3/29/2023 2:22|         67|    11725|      1|
|     24|http://facebook.com/|12/12/2022 11:41|3/19/2023 23:42|         28|     8404|      1|
|     61|http://www.facebo...| 3/22/2023 17:06|3/29/2023 22:30|         97|     3880|      1|
|     73|https://www.faceb...|  2/22/2023 4:14|3/29/2023 22:30|        133|     3325|      1|
|    318|https://web.faceb...| 9/13/2022 17:04| 3/27/2023 0:13|         22|     1223|      1|
|    470|https://web.faceb...| 11/23/2022 1:08|3/23/2023 17:27|         23|     1143|      1|
|    658|https://web.faceb...| 3/29/2022 19:24|2/18/2023 11:

In [21]:
web_history.createOrReplaceTempView("web_history")
average_frequency = web_history.selectExpr("AVG(frequency)").collect()[0][0]
query = web_history.filter((col("user_id") == 1) &
                           (col("url").like("%youtube%")) &
                           (col("frequency") > average_frequency)) \
                   .orderBy(col("frequency").desc())
query.show()

+-------+--------------------+---------------+---------------+-----------+---------+-------+
|hist_id|                 url|    first_visit|     last_visit|click_count|frequency|user_id|
+-------+--------------------+---------------+---------------+-----------+---------+-------+
|     11|https://www.youtu...|1/29/2023 14:33|3/25/2023 14:48|         24|    14792|      1|
|     69| http://youtube.com/|11/20/2022 6:00|2/21/2023 10:56|          9|     3430|      1|
|    108|https://www.youtu...|  3/6/2023 2:10| 3/6/2023 13:12|          3|     2344|      1|
|    217|https://www.youtu...|3/17/2023 15:05|3/17/2023 15:18|          2|     1716|      1|
|    226|http://www.youtub...| 12/4/2022 6:40| 2/6/2023 14:20|          5|     1658|      1|
|    297|https://www.youtu...| 2/6/2023 14:27|2/25/2023 11:05|         12|     1428|      1|
|    348|https://www.googl...| 3/11/2023 5:39| 3/11/2023 5:39|          1|     1144|      1|
|    345|https://www.googl...|3/11/2023 15:06|3/11/2023 15:06|        